In [1]:
from datasets import load_dataset
import numpy as np
import os 
import sys 
sys.path.append("..")

from src import constants
from src.datasets import mouse_dataloader_collate_fn
from torch.utils.data import DataLoader
import torch

from src.models.dwiseneurossm import DwiseNeuro


/workspace/sensorium-ssm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = load_dataset(constants.huggingface_dataset, split="train", streaming=True)
test_dataset = load_dataset(constants.huggingface_dataset, split="test", streaming=True)
dataloader_collate_fn = mouse_dataloader_collate_fn

train_dataloader = DataLoader(
    train_dataset,
    batch_size=3,  # Adjust as needed
    num_workers=0,  # Use multiple workers for speed
    collate_fn=lambda batch: dataloader_collate_fn(batch),
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=3,  # Adjust as needed
    num_workers=0,  # Use multiple workers for speed
    collate_fn=lambda batch: dataloader_collate_fn(batch),
)


In [3]:
model = DwiseNeuro(readout_outputs=[7440])
device = torch.device("cuda")
model.to(device)

DwiseNeuro(
  (core): DepthwiseCore(
    (stem): Sequential(
      (0): Conv3d(5, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (1): BatchNormAct(
        (bn): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): Identity()
      )
    )
    (blocks): Sequential(
      (0): PositionalEncoding3d()
      (1): InvertedResidual3d(
        (conv_pw): Sequential(
          (0): Conv3d(64, 384, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (1): BatchNormAct(
            (bn): BatchNorm3d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
        )
        (spat_covn_dw): Sequential(
          (0): Conv3d(384, 384, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), groups=384, bias=False)
          (1): BatchNormAct(
            (bn): BatchNorm3d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): SiLU(inpl

In [4]:
for batch in test_dataloader:
    inputs, targets = batch 
    print(inputs.shape)
    inputs, targets = inputs.to(device), targets.to(device)
    prediction = model(inputs)
    print(prediction)
    break

torch.Size([3, 5, 16, 64, 64])
[tensor([[[9.9049, 9.9072, 9.9099,  ..., 9.9075, 9.8987, 9.8979],
         [9.9355, 9.9354, 9.9340,  ..., 9.9305, 9.9283, 9.9353],
         [9.9233, 9.9236, 9.9232,  ..., 9.9287, 9.9283, 9.9223],
         ...,
         [9.8737, 9.8744, 9.8731,  ..., 9.8735, 9.8716, 9.8706],
         [9.8629, 9.8613, 9.8628,  ..., 9.8696, 9.8722, 9.8666],
         [9.9034, 9.8974, 9.9057,  ..., 9.8997, 9.9037, 9.9091]],

        [[9.9157, 9.9181, 9.9089,  ..., 9.9059, 9.9036, 9.9019],
         [9.9272, 9.9314, 9.9390,  ..., 9.9317, 9.9275, 9.9368],
         [9.9255, 9.9225, 9.9186,  ..., 9.9313, 9.9281, 9.9199],
         ...,
         [9.8678, 9.8714, 9.8731,  ..., 9.8811, 9.8795, 9.8757],
         [9.8702, 9.8707, 9.8715,  ..., 9.8658, 9.8666, 9.8636],
         [9.9059, 9.9012, 9.9086,  ..., 9.8989, 9.9058, 9.9061]],

        [[9.9093, 9.9098, 9.9056,  ..., 9.9103, 9.9032, 9.9014],
         [9.9331, 9.9359, 9.9330,  ..., 9.9333, 9.9352, 9.9387],
         [9.9233, 9.9248, 